<a href="https://colab.research.google.com/github/nataliaiglesiasbueno/UFV_visualizacion_datos/blob/main/clase_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Definición del mensaje que se quiere transmitir. Objetivos. Resultados. Conclusiones.

Usar gráficos de tipo exploratorio (porque todos conocemos los datos).

* Conocer los datos que apoyan al gráfico
* Conocer la persona a la que me voy a dirigir (al profesor)
* Conocer los limitante de tiempo, lugar, formato...

# **Datos**
* **COUNTRY** = país
* **SUBBRAND** = marco de producto
* **YEAR** = año
* **MONTH** = mes
* **SCENARIO** =
    * Actual significa que es una venta que se produjo real. forecast es una predicción que se hizo en otra fecha pero para esa fecha. Si es actual, forecast y forecast_year estarán vacíos. Estos campos indican cuando se hizo la predicción. P02 es enero, 3 es febrero...10 es septiembre (todo está con un +1 porque muchas empresas miden los meses como periodos (cada año tiene 13 periodos)).
    * Al_pf sería como el P13F; es decir, diciembre (lo haces a finales de diciembre para enero).
* **FORECAST**
* **FORECAST_YEAR**
* **AMOUNT** = datos de venta


Horizonte de previsión = cuantos puntos predices en cada ejecución. Tengo que saber desde cuando tengo forecast.

Necesito saber a partir de cuando tengo datos y para qué países.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving datos_ejercicio_ventas.csv to datos_ejercicio_ventas.csv


In [2]:
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt

nombres_columnas = ['COUNTRY', 'SUBBRAND','YEAR','MONTH','SCENARIO','FORECAST','FORECAST_YEAR','AMOUNT']

df = pd.read_csv('/content/datos_ejercicio_ventas.csv', header=None, skiprows=1)
df.columns = nombres_columnas

In [3]:
df.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


# **Limpieza base de datos**

In [6]:
forecast_counts = df['SCENARIO'].value_counts()

# Filtrar los valores de 'actuals' y 'AI_forecast'
forecast_counts[['actual', 'AI_forecast']]


,count
SCENARIO,
actual,900
AI_forecast,17766


In [8]:
paises_counts = df['COUNTRY'].value_counts()
paises_counts

,count
COUNTRY,
Great Britain,4008
Netherlands,2476
Hungary,2384
Denmark,2294
Spain,2152
Norway,2062
Czech,1403
Portugal,1179
Italy,708


In [9]:
productos_counts = df['SUBBRAND'].value_counts()
productos_counts

,count
SUBBRAND,
Pepsi Max (L3),5598
Pepsi Regular (L3),3636
7up (L3),2694
Lipton (L3),2693
7up Free (L3),2343
Mountain Dew (L3),1702


In [21]:
print(df.dtypes)

COUNTRY           object
SUBBRAND          object
YEAR               int64
MONTH              int64
SCENARIO          object
FORECAST          object
FORECAST_YEAR    float64
AMOUNT           float64
dtype: object


## **cambiar luego para graficas forecast para que se vean los meses en vez de lo de PF**

# **Horizonte de Previsión**

**NOTAS DE CLASE: TENGO QUE REDACTAR ESTO DESPUES**

Contexto: la variable scenario puede ser actual o AI_forecast. Actual significa que es una venta que se produjo real. forecast es una predicción que se hizo en otra fecha pero para esa fecha. Si es actual, forecast y forecast_year estarán vacíos. Estos campos indican cuando se hizo la predicción. P02 es enero, 3 es febrero...10 es septiembre (todo está con un +1 porque muchas empresas miden los meses como periodos (cada año tiene 13 periodos)).
    * Al_pf sería como el P13F; es decir, diciembre (lo haces a finales de diciembre para enero).

Primer fila: en enero de 2023 hice una previsión de lo que iba a vender el producto Lipton de Portugal en diciembre del 2023. el horizonte de prevision es el numero de puntos que tienes (desde la primera predicción hasta la final). todos los países tienen el mismo horizonte de prevision. hazme el código para el  país portugal

Voy a calcularlo para Portugal:

**COMPROBAR QUE ESTO ESTA BIEN ----> creo que no**



In [16]:
# Filtro los datos para Portugal
portugal_data = df[df['COUNTRY'] == 'Portugal']

# Cojo los registros donde el escenario es 'AI_forecast' ---> predicciones
portugal_forecasts = portugal_data[portugal_data['SCENARIO'] == 'AI_forecast']

# Contar cuántos puntos distintos de predicciones tiene Portugal, utilizando los valores únicos de las combinaciones de 'forecast_year' y 'forecast'
unique_forecast_points = portugal_forecasts[['FORECAST_YEAR', 'FORECAST']].drop_duplicates()

# Contar los puntos de predicción distintos
num_distinct_forecasts = unique_forecast_points.shape[0]

num_distinct_forecasts

12

Falta hacer: histórico de datos actuals/forecasts y falta los forecasts distintos

# **Distribución de las ventas de cada país (PRUEBAS)**

TENGO Q VER COMO SE DISTRIBUYEN LAS VENTAS POR LOS PAISES (SI LOS PRODUCTOS ESTAN BIEN DISTRIBUIDOS O ESTÁN PRINCIPALMENTE EN UN PAIS... VER SI TODAS ESTAN AL PRICNIPIO DEL AÑO O ESTAN DISTRIBUIDAS A LO LARGO DE TODO...)



Great Britain es el pais con mas ventas (amount se mide en litros)

In [20]:
import plotly.express as px

# Filtro los datos donde el escenario es actual ---> ventas reales
actual_sales = df[df['SCENARIO'] == 'actual']

fig = px.bar(
    actual_sales.groupby('COUNTRY')['AMOUNT'].sum().reset_index(),
    x='COUNTRY',
    y='AMOUNT',
    title='Distribución de Ventas Reales por País',
    labels={'country': 'País', 'amount': 'Ventas Totales'}
)

fig.show()

In [19]:
import plotly.express as px

fig = px.bar(
    actual_sales.groupby('SUBBRAND')['AMOUNT'].sum().reset_index(),
    x='SUBBRAND',
    y='AMOUNT',
    title='Distribución de Ventas Reales por Submarca',
    labels={'subbrand': 'Submarca', 'amount': 'Ventas Totales'}
)

fig.show()